In [1]:
class Value:
    def __init__(self, data, prev=[], operation=""):
        self.data = data
        self.grad = 0
        self._prev = prev
        self._operation = operation
        self._backward = lambda: None
    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad}), operation={self._operation}"
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        output = Value(self.data + other.data, [self, other], "+")
        def backward():
            self.grad += output.grad
            other.grad += output.grad
        output._backward = backward
        return output
    def __sub__(self, other):
        return self + (-other)
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        output = Value(self.data * other.data, [self, other], "*")
        def backward():
            self.grad += output.grad * other.data
            other.grad += output.grad * self.data
        output._backward = backward
        return output
    def __pow__(self, k):
        output = Value(self.data ** (k), [self], "pow")
        def backward():
            self.grad += output.grad * k * self.data**(k-1)
        output._backward = backward
        return output
    def __truediv__(self, other):
        return self * other**(-1)
    def __neg__(self):
        return self * Value(-1)
    def backward(self):
        visited = set()
        sorted = []
        def topSort(node):
            if node in visited:
                return
            for prev in node._prev:
                topSort(prev)
            sorted.append(node)
            visited.add(node)
        topSort(self)
        self.grad = 1
        for node in reversed(sorted):
            node._backward()

In [2]:
x1 = Value(0); y1 = Value(2)
x2 = Value(1); y2 = Value(5)

In [3]:
w = Value(1)
b = Value (-2)

lr = 0.1

for i in range(10):
    print("##### iteration " + str(i) + "#####")
    print("==forward pass")
    L = ((x1*w+b - y1)**2 + (x2*w+b - y2)**2)/2
    print("L=" + str(L))

    print("==backward pass")
    w.grad = 0;b.grad = 0
    x1.grad = 0; y1.grad = 0; x2.grad = 0; y2.grad = 0
    L.backward()
    print(w)
    print(b)

    print("==gradient descent")
    w.data += -lr * w.grad
    b.data += -lr * b.grad
    print(w)
    print(b)

##### iteration 0#####
==forward pass
L=Value(data=26.0, grad=0), operation=*
==backward pass
Value(data=1, grad=-6.0), operation=
Value(data=-2, grad=-10.0), operation=
==gradient descent
Value(data=1.6, grad=-6.0), operation=
Value(data=-1.0, grad=-10.0), operation=
##### iteration 1#####
==forward pass
L=Value(data=14.180000000000001, grad=0), operation=*
==backward pass
Value(data=1.6, grad=-4.4), operation=
Value(data=-1.0, grad=-7.4), operation=
==gradient descent
Value(data=2.04, grad=-4.4), operation=
Value(data=-0.2599999999999999, grad=-7.4), operation=
##### iteration 2#####
==forward pass
L=Value(data=7.737999999999998, grad=0), operation=*
==backward pass
Value(data=2.04, grad=-3.2199999999999998), operation=
Value(data=-0.2599999999999999, grad=-5.4799999999999995), operation=
==gradient descent
Value(data=2.362, grad=-3.2199999999999998), operation=
Value(data=0.28800000000000003, grad=-5.4799999999999995), operation=
##### iteration 3#####
==forward pass
L=Value(data=4.